In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime
import urllib.request
import urllib.error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import re
import time
%matplotlib inline

In [2]:
#Dictionary of team names and abbreviations. VEG is used instead of VGK.
teamAbvs = {'Anaheim Ducks':'ANA', 'Arizona Coyotes':'ARI', 'Atlanta Thrashers': 'ATL', 'Boston Bruins': 'BOS', 'Buffalo Sabres':'BUF', 'Carolina Hurricanes':'CAR',
            'Calgary Flames':'CGY', 'Chicago Blackhawks':'CHI', 'Columbus Blue Jackets':'CBJ', 'Colorado Avalanche':'COL', 'Dallas Stars':'DAL',
           'Detroit Red Wings':'DET', 'Edmonton Oilers':'EDM', 'Florida Panthers':'FLA', 'Los Angeles Kings':'LAK',
           'Minnesota Wild':'MIN', 'Montreal Canadiens':'MTL', 'Nashville Predators':'NSH', 'New Jersey Devils':'NJD',
           'New York Islanders':'NYI', 'New York Rangers':'NYR', 'Ottawa Senators':'OTT', 'Phoenix Coyotes':'PHX', 'Philadelphia Flyers':'PHI',
           'Pittsburgh Penguins':'PIT', 'San Jose Sharks':'SJS', 'St. Louis Blues':'STL', 'Tampa Bay Lightning':'TBL',
           'Toronto Maple Leafs':'TOR','Vancouver Canucks':'VAN', 'Vegas Golden Knights':'VEG', 'Washington Capitals':'WSH', 'Winnipeg Jets':'WPG',
           }

In [30]:
#Need to produce the table for each season
def startSeason(season_url):
    season_year = season_url.split('_')[1]
    
    #Retrieving the initial url and HTML.
    try:
        season_page = urlopen(season_url)
    except urllib.error.URLError:
        return print('Error with URL')
        
    season_soup = BeautifulSoup(season_page, 'lxml')
    
    #Building the schedule table.
    season_games = season_soup.find(id='games')
    Date =[]
    Vis=[]
    VG=[]
    Home=[]
    HG=[]
    OT=[]
    Att=[]
    LOG=[]
    Notes=[]

    #Create a table with a list for each column of data.
    headers = [Date, Vis, VG, Home, HG, OT, Att, LOG, Notes]
    
    #Iterate through each row and column, assigning the corresponding data to the location in 'headers'.
    for row in season_games.findAll('tr'):
        cells = row.findAll(['td','th'])
        index = np.arange(0,len(cells)).tolist()
        for num in index:
            headers[num].append(cells[num].find(text=True))
            
    #Create the games Dataframe, set the column names to names instead of index values.
    season_gamesDf = pd.DataFrame(headers).transpose()
    season_gamesDf.columns = season_gamesDf.iloc[0]
    season_gamesDf = season_gamesDf.reindex(season_gamesDf.index.drop(0))
    
    #Create game HTML string for every game.
    gameHTML = []
    for entry in season_gamesDf.index:
        abv = teamAbvs[season_gamesDf['Home'][entry]]
        daystr = ''.join(season_gamesDf['Date'][entry].split('-'))
        gamestr = (daystr+'0'+abv)
        gamepage = ('https://www.hockey-reference.com/boxscores/'+gamestr+'.html')
        gameHTML.append(gamepage)
    #Add the new gameHTML column to gamesDf.
    season_gamesDf['Game HTML'] = gameHTML
    
    #Remove any cancelled games
    season_gamesDf['Date'] = pd.to_datetime(season_gamesDf['Date'], infer_datetime_format=True)
    cancelledDf = season_gamesDf[(season_gamesDf['Att.'].isna() == True) & (season_gamesDf['Date'] < datetime.now())]
    cancelled_games = cancelledDf.index.tolist()
    season_gamesDf = season_gamesDf.reindex(season_gamesDf.index.drop(cancelled_games))
    
    return season_gamesDf

In [139]:
#Need to build team dfs for the season
teams = sorted(list(teamAbvs.values()))
all_teams = sorted(list(teamAbvs.values()))
#Need ATL and PHX in the dictionary for the HTML retrieval, do not want separate DFs for them.
teams.remove('ATL')
teams.remove('PHX')

#Column names
table_columns = ['otherteam', 'homegames', 'awaygames', 'ngames', 'homepenalties', 'awaypenalties', 'totalpenalties', 'normalizedpenalties']

#Dictionary with team dfs
teams_dict = dict.fromkeys(teams)

#Initial DF data creation
emptydata = np.zeros(len(teams)).tolist()
for t in teams:
    edf_data = dict.fromkeys(table_columns)
    edf_data['otherteam'] = teams
    for c in table_columns[1:]:
        edf_data[c] = emptydata
    df = pd.DataFrame(edf_data)
    teams_dict[t] = df

In [141]:
#Now need to open each page in the season_gamesDf
season_page = startSeason('https://www.hockey-reference.com/leagues/NHL_2018_games.html')
for num in season_page.index.tolist()[1156:]:
    #Open the page for a game
    gamepage = season_page['Game HTML'][num]
    try:
        openedgame = urlopen(gamepage)
    except urllib.error.URLError:
        print('Error with URL')
    #Gather the html and the penalty table
    gamesoup = BeautifulSoup(openedgame, 'lxml')    
    penaltytable = gamesoup.find(id='penalty')
    #Initialize necessary variables for each game
    hometeam = teamAbvs[season_page['Home'][num]]
    visitorteam = teamAbvs[season_page['Visitor'][num]]
    homepenalties = 0
    visitorpenalties = 0
    #Parse the penalties and record which team took penalties
    for row in penaltytable.findAll('tr'):
        rowtext = row.get_text().upper().split()
        penalizedteam = [name for name in rowtext if name in all_teams]
        name = ''
        if len(penalizedteam) > 0:
            name = penalizedteam[0]
            if name == 'PHX':
                name = 'ARI'
            if name == 'ATL':
                name = 'WPG'
        if name == hometeam:
            homepenalties += 1
        if name == visitorteam:
            visitorpenalties += 1
    #Add the new data to the homes table
    homedf = teams_dict[hometeam]
    indexvis = homedf[homedf['otherteam']==visitorteam].index[0]
    homedf.at[indexvis, 'homegames'] += 1
    homedf.at[indexvis, 'homepenalties'] += homepenalties
    
    #Add the new data to the visitors table
    visdf = teams_dict[visitorteam]
    indexhome = visdf[visdf['otherteam']==hometeam].index[0]
    visdf.at[indexhome, 'awaygames'] += 1
    visdf.at[indexhome, 'awaypenalties'] += visitorpenalties
    print(num)

1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
1200
1201
1202
1203
1204
1205
1206
1207
1208
1209
1210
1211
1212
1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1237
1238
1239
1240
1241
1242
1243
1244
1245
1246
1247
1248
1249
1250
1251
1252
1253
1254
1255
1256
1257
1258
1259
1260
1261
1262
1263
1264
1265
1266
1267
1268
1269
1270
1271


In [143]:
#Save all tables to their own csv file for importing into SQL database
for t in teams:
    tdf = teams_dict[t]
    tdf['ngames'] = tdf['homegames']+tdf['awaygames']
    tdf['totalpenalties'] = tdf['homepenalties']+tdf['awaypenalties']
    tdf['normalizedpenalties'] = tdf['totalpenalties']/tdf['ngames']
    tdf['normalizedhomepenalties'] = tdf['homepenalties']/tdf['homegames']
    tdf['normalizedawaypenalties'] = tdf['awaypenalties']/tdf['awaygames']
    #tdf.to_csv(t+'_penalties_20172018', index=False, header=True)